# 웹스크래핑  
네이버 종목 토론방에서 작성일자/제목(댓글 수도 제목에 포함)/조회수/추천수/비추천수를 수접함

In [ ]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
from selenium import webdriver # selenium 제어창에 대한 옵션
options=webdriver.ChromeOptions()
options.add_argument('--privileged')
options.add_argument('headless')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--start-maximized')
options.add_argument('disable-gpu')  # GPU 사용 안함
options.add_argument('lang=ko_KR')  # 언어 설정

In [ ]:
# 디렉토리 재설정
try:
    os.chdir("C:\\Users\\SAMSUNG\\Desktop\\") # Chromewebdriver가 있는 경로로 설정 
    print("Directory changed")
except OSError: # 예외처리
    print("Can't change the Current Working Directory")

In [ ]:
def mkdir_data(directory): # 하위 디렉토리 생성
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("Directory created")
    except OSError:
        print('Error: Creating directory.')

In [ ]:
def web_scraping(code, start_page,end_page, count): # 종목토론방 긁어오기
    title_result=[]
    date_result=[]
    like_result=[]
    dislike_result=[]
    views_result=[]

    driver=webdriver.Chrome("chromedriver.exe", options=options)
    url="https://finance.naver.com/item/board.nhn?code="+code+"&page="+str(start_page)
    driver.get(url)
    while (start_page<=end_page):
        if (start_page==1): # 첫 페이지
            page=start_page
        elif (start_page<=10): # 2~10 페이지, 밑의 페이지 bar 구성이 다름
            page=start_page+1
        else:
            if (start_page%10==0): # view의 마지막 페이지인 경우
                page=12
            else:
                page=start_page%10+2
        html = driver.page_source
        soup=bs(html,'lxml') # selenium의 각 화면에서 bs4를 이용하여 긁어옴
        
        # 각 요소를 끌어오는 부분
        title_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td.title > a"
        title=soup.select(title_css_selector)
        for i in title: 
            title_result.append(i.text.replace('\t','').replace('\n',''))
    
        date_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(1) > span"
        date=soup.select(date_css_selector)
        for j in date:
            date_result.append(j.text)
    
        views_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(4) > span"
        views=soup.select(views_css_selector)
        for k in views:
            views_result.append(k.text)
    

        like_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(5) > strong"
        like=soup.select(like_css_selector)
        for l in like:
            like_result.append(l.text)
    
        dislike_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(6) > strong"
        dislike=soup.select(dislike_css_selector)
        for m in dislike:
            dislike_result.append(m.text)
        
        # 자동으로 다음 새 10개 페이지로 이동하는 부분
        nextpg_css_selector=f"#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child({page+1}) > a"
        nextpg_element=driver.find_element_by_css_selector(nextpg_css_selector)
        nextpg_element.click()
        start_page+=1
    # 모인 요소들을 합치는 부분
    concating(code+'.KS', title_result,date_result,views_result,like_result,dislike_result,count)
    driver.quit()

In [ ]:
def concating(code, title_result,date_result,views_result,like_result,dislike_result,count): # 긁어온 데이터를 합치는 함수
    concat_data={'title':title_result,
                'date':date_result,
                'views':views_result,
                'like':like_result,
                'dislike':dislike_result}
    scraping_df=pd.DataFrame(concat_data)
    scraping_df['Code']=code
    scraping_df.to_csv('./title_scraping_'+code+'_'+str(count)+'.csv',mode='w',encoding='utf-8-sig',header=True,index=True)

In [ ]:
if __name__ == '__main__':
    code=input("기업 코드를 입력해주세요.: ")
    start_page=int(input("첫 번째 페이지를 입력해주세요.: "))
    end_page=int(input("마지막 페이지를 입력해주세요.: "))
    mkdir_data(code+'_data') # 디렉토리 생성
    count=1
    while (start_page<=end_page):
        if (end_page-start_page<100): # (1)
            web_scraping(code,start_page,end_page,count)
        else:
            web_scraping(code,start_page,start_page+100,count) # (2)
        print("%d번째 스크래핑이 끝났습니다."%count)
        start_page+=101 # (3), (1)~(3)의 숫자는 임의 조정 가능. 
        count+=1

# 파일 모으기  
여러개로 나누어진 종목토론방 데이터를 하나로 묶는다.

In [ ]:
import glob
import csv

In [ ]:
def merge_file(input_path, merge_output):
    file_list=glob.glob(os.path.join(input_path,'*'))
    with open(merge_output,'w',encoding='utf-8-sig') as f:
        for i, file in enumerate(file_list):
            if i==0: # 처음 파일에만 헤더 포함!
                with open(file,'r',encoding='utf-8-sig') as f2:
                    while True:
                        line=f2.readline()
                        
                        if not line:
                            break
                        f.write(line)
                file_name=file.split('\\')[-1]
                print(file_name+ ' complete')
            else:
                with open(file,'r',encoding='utf-8-sig') as f2:
                    n=0
                    while True:
                        line=f2.readline()
                        if not line:
                            break
                        if n!=0:
                            f.write(line)
                        n+=1
                file_name=file.split('\\')[-1]
                print(file_name+ ' complete')
    print("모든 파일 완료")

In [ ]:
input_path=input("경로를 입력해주세요. :") # 기업별 하위 디렉토리 경로 ./기업코드_data 형식
input_path.replace("\\","\/") # 복사 붙여넣기 해도 \를 /로 바꿔준다.
input_path=input_path+'\/' # 맨 마지막에도 채워 넣어 준다.
output_name=input("최종 파일 이름을 입력해주세요. :")
output_path=input("최종 파일 경로를 입력해주세요. :")
output_path.replace("\\","\/")
merge_output=output_path+'\\'+output_name+'.csv'
merge_file(input_path,merge_output)

# 형태소 분석  
모인 데이터를 바탕으로 형태소를 분석한다.

In [ ]:
import rhinoMorph # 형태소 분석기
import re
rn = rhinoMorph.startRhino()

In [ ]:
def Morph(df):
    print(df.isna().sum()) # 결측치 확인
    comment(df)
    
    morphed_data=''

    for i in df['adj_title']:
        morphed_data_object=rhinoMorph.onlyMorph_list(rn,i,pos=['NNG', 'VCP', 'VCN', 'MAG', 'VA', 'VV', 'XR','MM','NV','NF'],eomi=True)
        joined_data_object=' '.join(morphed_data_object)
    
    if joined_data_object:
        morphed_data+=joined_data_object+'\n'
        
    merge_text_list=re.split('\n| ',morphed_data)
    word=pd.Series(merge_text_list)
    result=word.value_counts()
    
    result.to_csv('result.csv',mode='w',encoding='utf-8-sig',header=True,index=True)

In [ ]:
def comment(df): # 제목에서 댓글 수 뽑아내기. 댓글은 [n]의 형태
    df['comment']=0
    df['adj_title']=''
    
    k=0
    for j in df['title']:
        if j:
            if j[-1]==']': 
                df['comment'][k]=float(j[-2])
                df['adj_title'][k]=j[:-3]
            else:
                df['adj_title'][k]=j
        k+=1
        
    df['date']=pd.to_datetime(df['date'],format='%Y.%m.%d %H:%M') # 문자열로 받은 날짜 데이터 포맷 변경
    df.drop(labels=['Unnamed: 0','title'],axis=1,inplace=True) # 원래 타이틀은 댓글 수를 포함하고 있으므로 분리 이후에 삭제
    
    return df

In [ ]:
data=pd.read_csv('./merge_output.csv', encoding='utf-8-sig') # 파일 이름은 merge_file 함수를 거치고 난 결과물을 대입
Morph(data)

# Levenshtein Algorithm  
형태소 분리 된 데이터에 대해 편집거리 알고리즘을 적용한다.  
levenshtein을 두 개로 분리한 이유? = 파이썬의 for문 호출 제한 이슈 때문

In [ ]:
from jamo import h2j, j2hcj # 자모 분리 라이브러리

In [ ]:
def change_cost(a,b): # 교체비용에 해당하는 함수
    if a==b: # 두 문자가 아예 같을 경우
        return 0
    return round(levenshtein(j2hcj(h2j(a)),j2hcj(h2j(b)))/3,2) # 다르다면 자모 단위까지 levenshtein을 적용한다.

In [ ]:
def jamo_levenshtein(a, b): # 자모단위 levenshtein
    al=len(a)
    bl=len(b)
    if a==b: #문자열이 같을 때
        return 0
    if al<bl:
        return jamo_levenshtein(b,a)
    if b=='': #두 번째 문자열이 공백일 경우
        return al
    
    matrix=np.zeros((al+1,bl+1)) #모든값이 0인 행렬 반환
    
    for i in range(al+1): #초기화
        matrix[i][0]=i
    for j in range(bl+1):
        matrix[0][j]=j
    
    for i in range(1, al+1):
        aw=a[i-1]
        for j in range(1, bl+1):
            bw=b[j-1]
            matrix[i][j]=min({
                matrix[i-1][j]+1, #삭제
                matrix[i][j-1]+1, #삽입
                matrix[i-1][j-1]+(aw!=bw) #다를경우 1추가
            })
    return matrix[-1][-1] #행렬의 오른쪽 최하단값 반환

In [ ]:
def levenshtein(a, b): # 글자단위 levenshtein
    al=len(a)
    bl=len(b)
    if a==b: #문자열이 같을 때
        return 0
    if al<bl:
        return levenshtein(b,a)
    if b=='': #두 번째 문자열이 공백일 경우
        return al
    
    matrix=np.zeros((al+1,bl+1)) #모든값이 0인 행렬 반환
    
    for i in range(al+1): #초기화
        matrix[i][0]=i
    for j in range(bl+1):
        matrix[0][j]=j
    
    for i in range(1, al+1):
        aw=a[i-1]
        for j in range(1, bl+1):
            bw=b[j-1]
            matrix[i][j]=min({
                matrix[i-1][j]+1, #삭제
                matrix[i][j-1]+1, #삽입
                matrix[i-1][j-1]+change_cost(aw,bw) #다를경우 1추가
            })
    return matrix[-1][-1] #행렬의 오른쪽 최하단값 반환

In [ ]:
data=pd.read_csv('C:\\Users\\SAMSUNG\\Downloads\\전체형태소_삼성전자.csv')
data.rename(columns={'Unnamed: 0':'형태소', '0':'빈도수','Unnamed: 2':'감성점수','Unnamed: 3':'비고'},inplace=True)
data

In [ ]:
data_low_frequency=data[(data.빈도수<20)&(data.빈도수>17)] # 숫자는 상황에 따라 임의 조정
data_with_sent=data[data.빈도수>=20]

In [ ]:
word_dict={}
result=[]

for base in data_low_frequency['형태소']:
    for source in data_with_sent['형태소']:
        if levenshtein(base,source)<0.5:
            word_dict[source]=levenshtein(base,source)
    print(base)
    print(word_dict)
    
    if any(word_dict): # 만약 가까운 요소가 출력이 안된다면 base를 그대로 사용
        result.append(base)
        continue

    ans=input("출력된 단어 중 가장 가까운 단어를 골라 써주세요(없으면 x입력): ")
    if ans in word_dict:
        result.append(ans)
        print("단어가 대체되어 등록되었습니다.")
    else if ans=='x':
        print("선택하지 않으셨습니다.")
        result.append(base) # 출력된 단어 중 어느것과도 가깝지 않으면 base를 그대로 사용
    else:
        print("찾을 수 없는 단어입니다.")
        
    word_dict={} #초기화
    
data_low_frequency['형태소']=result # 바뀐 값 복사

# 주가 데이터 불러오기(일단위)  
pandas-datareader사용, 최근 5년의 주가데이터를 불러온다.  
pandas-datareader를 사용하는 이유? = 야후 사이트만이 수정종가를 제공하기 때문

In [ ]:
import pandas_datareader.data as web
from datetime import datetime

In [ ]:
code={} # 코드 딕셔너리를 불러와 Datareader 인수로 전달한다.
code=code_dict(code)

In [ ]:
for stock_code, stock_name in code.items():
    stock_data=web.DataReader(stock_code,'yahoo')
    stock_data['기업명']=stock_name
    stock_data.to_csv(stock_name+'_alltime_주가데이터.csv',encoding='utf-8-sig')

print('작업완료')

# 주가 데이터 불러오기(실시간)  
네이버 금융에서 실시간으로 변하는 주가를 불러오는 부분.  
종목 토론방의 데이터를 긁어오는 매커니즘과 비슷하다.

In [ ]:
def stock_scraping(code, stock_date,stock_time):
    time_result=[]
    price_result=[]
    pivot=1
    
    driver=webdriver.Chrome("chromedriver.exe", options=options)
    url='https://finance.naver.com/item/sise_time.nhn?code='+code+'&thistime='+str(stock_date)+str('stock_time')
    driver.get(url)
    
    while True:
        if pivot==1:
            page=pivot
        elif(pivot<=10):
            page=pivot+1
        else:
            if(pivot%10==0):
                page=12
            else:
                page=(pivot%10)+2
            
        html = driver.page_source
        soup=bs(html,'lxml')

        time_css_selector="body > table.type2 > tbody > tr > td:nth-of-type(1) > span"
        now_time=soup.select(time_css_selector)
        for i in now_time:
            time_result.append(i.text)
    
        price_css_selector="body > table.type2 > tbody > tr > td:nth-of-type(2) > span"
        price=soup.select(price_css_selector)
        for j in price:
            price_result.append(j.text)
        
        nextpg_css_selector=f"body > table.Nnavi > tbody > tr > td:nth-child({page+1}) > a"    
        try:
            nextpg_element=driver.find_element_by_css_selector(nextpg_css_selector)
            nextpg_element.click()
        except NoSuchElementException:
            print("마지막 페이지")
            break
        pivot+=1

    concating(code, time_result,price_result,stock_time)
    driver.quit()

In [ ]:
def concating(code, time_result, price_result, stock_time):
    date_result=stock_date
    concat_data={'date':date_result,
                 'time':time_result,
                'price':price_result}
    scraping_df=pd.DataFrame(concat_data)
    scraping_df.to_csv('./stock_crwaling_'+code+'_'+str(date_result)+'.csv',mode='w',encoding='utf-8-sig',header=True,index=True)

In [ ]:
code=input("기업 코드를 입력해주세요.: ")
stock_date=int(input("날짜를 입력해주세요.(예시: 20210207): "))
stock_time=int(input("시간을 입력해주세요.(예시: 161059): "))
stock_scraping(code,stock_date,stock_time)
print("완료")

# 기업코드 매칭  
(기업코드6자리).KS 형태로 저장되어 있는 데이터들을 기업 코드에 맞는 기업 명으로 매칭하는 작업  

In [ ]:
def code_dict(dictionary): # 코드-기업명 딕셔너리 생성.
    data=open('기업코드.txt','r',encoding='UTF8')
    line=data.readlines()
    data.close()
    code_key=[]
    code_value=[]
    for i in line:
        temp=i[:-1] # 줄바꿈 제거
        code_key.append(temp.split(" ")[1]+'.KS') # 공백기준 코드만 뽑기
        code_value.append(temp.split(" ")[0]) # 공백기준 기업명 뽑기
    code=dict(zip(code_key,code_value)) # zip은 인덱스 기준으로 리스트를 순서대로 가져오는 함수, dict는 딕셔너리 변환
    return code    

In [ ]:
def code_matching(df,code):
    code_key_list=list(code.keys())
    code_name_list=list(code.values())
    match_code=[]
    for i, data in enumerate(df['Code']):
        for j, obj in enumerate(code_key_list):
            if data==obj:
                match_code.append(code_name_list[j])
    df['기업명']=match_code